# File for training the Faster RCNN Model and saving it

In [ ]:
from torchvision import tv_tensors
import torch
from tqdm import tqdm
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import xml.etree.ElementTree as ET
from google.colab import drive
import os
import pandas as pd
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.transforms import v2 as T

drive.mount('/content/mydrive', force_remount=True)

In [ ]:
# Custom dataset to load in data in desired format for Faster RCNN model
class CustomDataset(Dataset):
    def __init__(self, data_dir, target_size=(1280, 720)):
        self.target_size = target_size
        self.data_dir = data_dir
        self.image_paths = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = os.path.join(self.data_dir, self.image_paths[idx])
        image = Image.open(image_path).convert("RGB")
        image = image.resize(self.target_size, Image.BILINEAR)
        csv_filename, _ = os.path.splitext(self.image_paths[idx])
        csv_path = os.path.join(self.data_dir, csv_filename + '.csv')


        df = pd.read_csv(csv_path)

        boxes_list = []
        labels_list = []
        width = df.iloc[0]['width']
        height = df.iloc[0]['height']


        for index, row in df.iterrows():
            class_label = row['class']
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']

            xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)

            boxes_list.append([xmin, ymin, xmax, ymax])
            labels_list.append(class_label)

        boxes = torch.as_tensor(boxes_list, dtype=torch.float32)
        labels = torch.tensor([labels_list.index(l) for l in labels_list], dtype=torch.int64)
        targets = []
        for i in range(len(boxes_list)):
            target = {
                'boxes': boxes[i],
                'labels': labels[i],
                'image_id': torch.tensor([idx]),  # Add image ID for multiclass detection
                'area': (boxes[i, 3] - boxes[i, 1]) * (boxes[i, 2] - boxes[i, 0]),
                'iscrowd': torch.tensor([0], dtype=torch.int64)
            }
            targets.append(target)
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])


        img, target = transform(img, target)

        return  img, target

Mounted at /content/mydrive


In [ ]:
data_dir = '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset'
annotation_paths = []
# List all files in the directory
files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f)) and f.endswith('.csv')]

In [ ]:
batch_size = 10
num_files = len(files)

for i in tqdm(range(0, num_files,batch_size)):
        batch_files = files[i:i + batch_size]
        for filename in batch_files:
            if filename.endswith('.csv'):
                annotation_paths.append(os.path.join(data_dir, filename))
print(annotation_paths)

100%|██████████| 1230/1230 [00:00<00:00, 69407.55it/s]

['/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/f66aa5f00ba96133629d46a6a6db09e8.csv', '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/f5d35995b3a01a2178ee6e6541f1d5c8.csv', '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/f5e6d361dc9398399eff41ef397f2425.csv', '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/f5f01d44ecd56b5b3253eb35c266870e.csv', '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/f64dc4d86868c6440e5f2b44cc4aea6c.csv', '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/f632da330108c076e77dfbab1793dbf0.csv', '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/f5d25f33689e16803b1d9975f4387873.csv', '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/archive/dataset/f5c9daf55818c17de2e8999682108eab.csv', '/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDe

In [ ]:
imgFiles = []
boxes = []
labels = []
count = 0
for f in tqdm(annotation_paths):
    count += 1
    df = pd.read_csv(f)
    for index, row in df.iterrows():
      class_label = row['class']
      xmin = row['xmin']
      ymin = row['ymin']
      xmax = row['xmax']
      ymax = row['ymax']
      boxes.append([xmin, ymin, xmax, ymax])
      labels.append(class_label)
      imgFiles.append(row['filename'] + ".jpg")


print(len(boxes))
print(len(labels))
print(len(imgFiles))


100%|██████████| 12291/12291 [00:48<00:00, 251.46it/s]

19746
19746
19746


In [ ]:
def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
class CustomDatasetCur(Dataset):
    def __init__(self, img_files, boxes, labels,transforms, target_size=(224, 224)):
        self.img_files = img_files
        self.boxes = boxes
        self.labels = labels
        self.target_size = target_size
        self.transforms = transforms

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):

        get_true = True
        while get_true:
          img_path = self.img_files[idx]
          box = self.boxes[idx]
          label = self.labels[idx]

          try:
            image = Image.open(os.path.join(data_dir, img_path)).convert("RGB")
            image = image.resize(self.target_size, Image.BILINEAR)
            idx = idx + 1
            get_true = False
          except:
            get_true = True

        # plt.imshow(image)
        # Convert box to tensor - important to include
        box = torch.tensor(box, dtype=torch.float32).to('cuda')
        lab = enc.fit_transform([label])

        # Note tensor data type matters
        target = {
            'boxes': box,
            'labels': torch.tensor(lab, dtype=torch.int64).squeeze().to(device),
            'image_id': torch.tensor([idx]).to(device),  # Add image ID for multiclass detection
            'area': torch.tensor([((box[2] - box[0]) * (box[3] - box[1]))]),
            'iscrowd': torch.tensor([0], dtype=torch.int64).to(device)
        }


        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        img = self.transforms(transform(image))
        return img, target

In [ ]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # Added this line for in cell testing

model = fasterrcnn_resnet50_fpn(pretrained=True)

#modify model output based on our classes
num_classes = 43 # Determined in classification with CNNS
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


trans = get_transform(train=True)
dataset = CustomDatasetCur(imgFiles, boxes,labels, transforms = trans)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.05, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# External pytorch content from pytorch docs - included when needed

# %cd mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")
# %cd ..
# %cd ..
# %cd ..

/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection
/content/mydrive/MyDrive/DeepLearning
/content/mydrive/MyDrive
/content/mydrive


In [ ]:
for epoch in range(20):
    print(f"Starting epoch {epoch}:")
    for index, en in tqdm(enumerate(data_loader),total=len(data_loader)):
        images, targets = en
        targets = [{k: v for k, v in t.items()} for t in [targets]]

        optimizer.zero_grad()
        output = model(images.to("cuda"), targets)  # This line will vary based on your model input
        loss = output['loss_classifier']  # Replace this with your actual loss calculation
        loss.backward()
        optimizer.step()

        if index % 100 == 0:
            print(f"Loss at index {index}: {loss.item()}")
            torch.save(model.state_dict(), f'/content/mydrive/MyDrive/DeepLearning/MilitaryObjectDetection/models/faster_rcnn_model_{epoch}.pth')
            print(output)

Starting epoch 0:


  0%|          | 0/19746 [00:00<?, ?it/s]

Loss at index 0: 0.0028090225532650948


  0%|          | 3/19746 [00:00<1:19:22,  4.15it/s]

{'loss_classifier': tensor(0.0028, device='cuda:0', grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0., device='cuda:0', grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.3532, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.2359, device='cuda:0', grad_fn=<DivBackward0>)}


...

 17%|█▋        | 3300/19746 [07:09<26:45, 10.25it/s]

Loss at index 3300: 2.523877640214778e-07


 17%|█▋        | 3302/19746 [07:10<50:22,  5.44it/s]

{'loss_classifier': tensor(2.5239e-07, device='cuda:0', grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0., device='cuda:0', grad_fn=<DivBackward0>), 'loss_objectness': tensor(4.1632, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(117.4311, device='cuda:0', grad_fn=<DivBackward0>)}


 17%|█▋        | 3366/19746 [07:17<32:10,  8.49it/s]